In [2]:
!pip install pdpipe

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for decore: filename=decore-0.0.1-py2.py3-none-any.whl size=4190 sha256=8aa9e9a4b17158cb9b8fc944f1b0fb0a584c32f7eabbd30b9d5be7c5e0f3f295
  Stored in directory: /root/.cache/pip/wheels/6c/1b/9c/71fa8b5df01cc605c4b5cadcd81df76d520149ca5c02b4013b
Successfully built decore


# Pipelining using pandas pdpipe.

In [0]:
import pandas as pd
import numpy as np
import pdpipe as pdp

In [0]:
df = pd.read_csv("/content/datasets_4909_7459_USA_Housing.csv")

In [5]:
df.head(2)

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."


In [7]:
df.shape

(5000, 7)

In [8]:

df.columns

Index(['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population', 'Price', 'Address'],
      dtype='object')

In [0]:
def size(n):
    if n<=4:
        return 'Small'
    elif 4<n<=6:
        return 'Medium'
    else:
        return 'Big'
df['House_size'] = df['Avg. Area Number of Rooms'].apply(size)

In [10]:
df.head(2)

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",Big
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA...",Big


# **Chaining the stages and doing the following operations - Droping the Age Column and Hot Encoding te House_Size column using Pipeline**

In [21]:
def price(x):
    if x>250000:
        return 'keep'
    else:
        return 'drop'

pipe = pdp.ColDrop('Avg. Area House Age')
pipe+= pdp.OneHotEncode('House_size')
pipe+=pdp.ApplyByCols('Price',price,'Price_tag',drop=False)
pipe+=pdp.ValDrop(['drop'],'Price_tag')
pipe+= pdp.ColDrop('Price_tag')
df4 = pipe(df)
df4.head(4)


,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size_Medium,House_size_Small
0,79545.458574,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",0,0
1,79248.642455,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA...",0,0
2,61287.067179,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482...",0,0
3,63345.240046,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820,1,0


In [22]:
df4.shape

(4990, 8)

# Scikit Learn for Scaling

In [23]:
pipeline_scale = pdp.Scale('StandardScaler',exclude_columns=['House_size_Medium','House_size_Small'])
df5 = pipeline_scale(df4)
df5.head(5)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size_Medium,House_size_Small
0,1.028113,0.019595,0.087245,-1.322810,-0.500532,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",-0.443768,-0.024527
1,1.000175,-0.257485,-0.722671,0.401923,0.775998,"188 Johnson Views Suite 079\nLake Kathleen, CA...",-0.443768,-0.024527
2,-0.690443,1.516179,0.929559,0.069730,-0.500662,"9127 Elizabeth Stravenue\nDanieltown, WI 06482...",-0.443768,-0.024527
3,-0.496719,-1.396285,-0.584986,-0.189886,0.075327,USS Barnett\nFPO AP 44820,2.253432,-0.024527
4,-0.813263,0.845954,0.200634,-0.992999,-1.723449,USNS Raymond\nFPO AE 09386,-0.443768,-0.024527


# NLTK Tokernizer Stage


In [28]:
pip_token = pdp.TokenizeText('Address')
df6 = pip_token(df5)
df6.head(5)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,House_size_Medium,House_size_Small
0,1.028113,0.019595,0.087245,-1.322810,-0.500532,"[208, Michael, Ferry, Apt, ., 674, Laurabury, ...",-0.443768,-0.024527
1,1.000175,-0.257485,-0.722671,0.401923,0.775998,"[188, Johnson, Views, Suite, 079, Lake, Kathle...",-0.443768,-0.024527
2,-0.690443,1.516179,0.929559,0.069730,-0.500662,"[9127, Elizabeth, Stravenue, Danieltown, ,, WI...",-0.443768,-0.024527
3,-0.496719,-1.396285,-0.584986,-0.189886,0.075327,"[USS, Barnett, FPO, AP, 44820]",2.253432,-0.024527
4,-0.813263,0.845954,0.200634,-0.992999,-1.723449,"[USNS, Raymond, FPO, AE, 09386]",-0.443768,-0.024527


In [31]:
def extract_state(token):
    return str(token[-2])

pipestate = pdp.ApplyByCols('Address',extract_state,result_columns='State')
df7 = pipestate(df6)
df7.head(5)

,Avg. Area Income,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,State,House_size_Medium,House_size_Small
0,1.028113,0.019595,0.087245,-1.322810,-0.500532,NE,-0.443768,-0.024527
1,1.000175,-0.257485,-0.722671,0.401923,0.775998,CA,-0.443768,-0.024527
2,-0.690443,1.516179,0.929559,0.069730,-0.500662,WI,-0.443768,-0.024527
3,-0.496719,-1.396285,-0.584986,-0.189886,0.075327,AP,2.253432,-0.024527
4,-0.813263,0.845954,0.200634,-0.992999,-1.723449,AE,-0.443768,-0.024527
